In [ ]:
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import numpy as np
import math
import os.path, os, sys, glob, json, pathlib
import jinja2
from multiprocessing import Pool

plt.rcParams.update({'font.size': 16, 'figure.figsize': [10.0, 0.08]})

def generate_from_template(templatefile, outfile, **kwargs):
    with open(templatefile, 'r') as infile, open(outfile, 'w') as outfile:
        output = jinja2.Template(infile.read()).render(kwargs)
        outfile.write(output)
        
faunusdir = '$HOME/SI-asynuclein-protonation/faunus'
workdir = '$HOME/SI-asynuclein-protonation/wildtype-0mM'
workdir

In [ ]:
%cd $workdir

numcpu=32
runeq=True
runprod=True
rerun=True
pH_range = np.arange(2, 8, 0.25)  # 2,8,0.25

def createRunDirectories(prefix, templatefile):
    ''' create run directories and eq + prod input files '''
    directory_list = [] # names of all run directories
    templatefile = os.path.abspath(templatefile)
    for pH in pH_range: # generate run directories w. input files
        dirname = '{}-pH{}'.format(prefix, pH)
        pathlib.Path(dirname).mkdir(exist_ok=True)
        directory_list.append(os.path.abspath(dirname))
        %cd -q $dirname
        generate_from_template(templatefile, 'eq.yml', pH=pH, micro=5000) # 5000
        generate_from_template(templatefile, 'input.yml', pH=pH, micro=100000) # 100000
        %cd -q ..
    return directory_list

def directoriesToRun(directory_list):
    ''' determine if directory needs to be run '''
    mask = []
    for name in directory_list:
        if rerun==True or (not os.path.isfile(name+'/out.json')): # run only if there's no state file
            mask.append(True)
        else:
            mask.append(False)
    return np.array(directory_list)[mask] # switch to numpy as lists cannot be masked

def launch_simulation(directory_name):
    ''' this will be executed on each node '''
    if os.path.isdir(directory_name):
        %cd -q $directory_name
        if rerun==True or (not os.path.isfile('out.json')): # run only if there's no state file
            if runeq or (not os.path.isfile('state.json')): # if state.json isn't there, run anyway
                print('o', end='', flush=True) # symbol for eq. run
                log = !$faunusdir/scripts/yason.py eq.yml | $faunusdir/faunus --nofun --nobar -v9 -o eqout.json
                with open('log_eq', 'w') as f:
                    for i in log:
                        f.write("%s\n" % i)
                    f.close()

            if runprod:
                print('O', end='', flush=True) # symbol for prod. run
                log = !$faunusdir/scripts/yason.py input.yml | $faunusdir/faunus --nofun --nobar -v9 -s state.json
                with open('log', 'w') as f:
                    for i in log:
                        f.write("%s\n" % i)
                    f.close()
        %cd -q ../
        
monomer_dirs = createRunDirectories('monomer', 'input-monomer.yml') # generate run directories for monomer
decamer_dirs = createRunDirectories('decamer', 'input-decamer.yml') # generate run directories for decamer
all_dirs = monomer_dirs + decamer_dirs # all generated directories
run_dirs = directoriesToRun(all_dirs)  # filtered to include only those dirs that needs to be (re)run

print("Number of jobs to run = {} out of {} total".format(len(run_dirs), len(all_dirs)))

with Pool(numcpu) as p: # split to multiple threads
    p.map(launch_simulation, run_dirs)

print('\ndone.')